In [2]:
import ctypes, win32gui, win32ui
from PIL import Image, ImageGrab

import time
time.sleep(10)
def get_cursor():
    hcursor = win32gui.GetCursorInfo()[1]
    hdc = win32ui.CreateDCFromHandle(win32gui.GetDC(0))
    hbmp = win32ui.CreateBitmap()
    hbmp.CreateCompatibleBitmap(hdc, 36, 36)
    hdc = hdc.CreateCompatibleDC()
    hdc.SelectObject(hbmp)
    hdc.DrawIcon((0,0), hcursor)
    
    bmpinfo = hbmp.GetInfo()
    bmpstr = hbmp.GetBitmapBits(True)
    cursor = Image.frombuffer('RGB', (bmpinfo['bmWidth'], bmpinfo['bmHeight']), bmpstr, 'raw', 'BGRX', 0, 1).convert("RGBA")
    
    win32gui.DestroyIcon(hcursor)    
    win32gui.DeleteObject(hbmp.GetHandle())
    hdc.DeleteDC()


    pixdata = cursor.load()


    width, height = cursor.size
    for y in range(height):
        for x in range(width):

            if pixdata[x, y] == (0, 0, 0, 255):
                pixdata[x, y] = (0, 0, 0, 0)


    hotspot = win32gui.GetIconInfo(hcursor)[1:3]

    return (cursor, hotspot)

cursor, (hotspotx, hotspoty) = get_cursor()
cursor.save("cursor.png")


ratio = ctypes.windll.shcore.GetScaleFactorForDevice(0) / 100

img = ImageGrab.grab(bbox=None, include_layered_windows=True)

pos_win = win32gui.GetCursorPos()
pos = (round(pos_win[0]*ratio - hotspotx), round(pos_win[1]*ratio - hotspoty))


img.paste(cursor, pos, cursor)

img.save("screenshot.png")

In [ ]:
import time
from fastapi import FastAPI
from starlette.responses import StreamingResponse
import nest_asyncio
nest_asyncio.apply()
app = FastAPI()
import asyncio

async def generate_data():



    while True:
        if yes==False:
            await asyncio.sleep(0.01)
        else:
                
            yield b"11\n"
            await asyncio.sleep(7)  # Continue sending True every 7 seconds
async def yes():
    global yes
    yes = False
    two_minutes = 60 * 2.5  # Convert minutes to seconds
    # two_minutes = 60 * 0.1
    await asyncio.sleep(two_minutes)
    yes = True
@app.get("/stream")
async def stream_data():
    return StreamingResponse(generate_data(), media_type="text/plain")

if __name__ == "__main__":
    import uvicorn
    asyncio.create_task(yes())
    uvicorn.run(app, host="localhost", port=9999)


INFO:     Started server process [10880]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:9999 (Press CTRL+C to quit)


INFO:     ::1:51235 - "GET /stream HTTP/1.1" 200 OK
INFO:     ::1:51286 - "GET /stream HTTP/1.1" 200 OK
INFO:     ::1:51297 - "GET /stream HTTP/1.1" 200 OK
